In [ ]:
import os
import numpy as np
import pandas as pd
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
print("축하해요 chughahaeyo you import all the necessary library!!!!")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


축하해요 chughahaeyo you import all the necessary library!!!!


In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
#load dataset
data=pd.read_csv('twitter_training.csv')
data= data.dropna()
data= data.sample(frac=1).reset_index(drop=True)
display(data)


,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,4447,Google,Neutral,Looking through my photos on Google and seeing...
1,9980,PlayStation5(PS5),Neutral,So For sale on eBay along well with several ot...
2,7442,LeagueOfLegends,Irrelevant,Thanks for the next mention after DragonesAcecho.
3,217,Amazon,Neutral,"""Amazon still running as usual for the UK reco..."
4,11173,TomClancysGhostRecon,Negative,@ GhostRecon FYI. I cannot get in. Error code ...
...,...,...,...,...
73990,9394,Overwatch,Negative,Im so bored. And overwatch h is pissing me off...
73991,12922,Xbox(Xseries),Positive,I'm so pumped for the.. I can't wait to see mo...
73992,4581,Google,Positive,"If you're stuck at home, you can play a game I..."
73993,10170,PlayerUnknownsBattlegrounds(PUBG),Positive,@PUBGMOBILE why are u doing campaign in pakist...


In [ ]:
df=data.iloc[0:10000,2:4]
df.columns=['class','text']
names=sorted(df['class'].unique().tolist())
N=list(range(len(names)))
normal_mapping=dict(zip(names,N)) #normal_mapping maps class names to numeric indices.
reverse_mapping=dict(zip(N,names)) #reverse_mapping maps numeric indices back to class names.
df['class2']=df['class'].map(normal_mapping)
print(df['class'].value_counts()) #This prints the count of each unique class in the class column.

class
Negative      3037
Positive      2836
Neutral       2444
Irrelevant    1683
Name: count, dtype: int64


In [ ]:
vader_analyzer = SentimentIntensityAnalyzer() #This initializes the VADER sentiment analyzer, which will be used to calculate sentiment scores for each text.

pos=[] # lists will store the sentiment scores for each text in the text column of the DataFrame.
neu=[]
neg=[]
comp=[]
for text in df['text']:  #This loop iterates over each text in the text column of the DataFrame and calculates its sentiment scores using the VADER sentiment analyzer
    score = vader_analyzer.polarity_scores(text)
    pos+=[score['pos']]
    neu+=[score['neu']]
    neg+=[score['neg']]
    comp+=[score['compound']]

df['positive']=pos
df['neutral']=neu
df['negative']=neg
df['comp']=comp
display(df)

,class,text,class2,positive,neutral,negative,comp
0,Neutral,Looking through my photos on Google and seeing...,2,0.146,0.854,0.000,0.5574
1,Neutral,So For sale on eBay along well with several ot...,2,0.162,0.838,0.000,0.7003
2,Irrelevant,Thanks for the next mention after DragonesAcecho.,0,0.326,0.674,0.000,0.4404
3,Neutral,"""Amazon still running as usual for the UK reco...",2,0.100,0.762,0.138,-0.3182
4,Negative,@ GhostRecon FYI. I cannot get in. Error code ...,1,0.333,0.667,0.000,0.7900
...,...,...,...,...,...,...,...
9995,Negative,Nice one to know why you Vegas guys aren've t ...,1,0.124,0.620,0.256,-0.7450
9996,Positive,This!!!!! ALLLLL OF U,3,0.000,1.000,0.000,0.0000
9997,Irrelevant,"all those doing this, take it go. You lying on...",0,0.083,0.799,0.118,-0.2500
9998,Positive,nah I like you better,3,0.692,0.128,0.179,0.6124


In [ ]:
X=df.iloc[:,3:7]
Y=df.iloc[:,2]

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
columns=X.columns.to_list() #This converts the column names of the DataFrame X to a list.
preds = np.zeros((testX.shape[0])) #This initializes an array of zeros with the same number of elements as the number of rows in testX.
kf = KFold(n_splits=5,random_state=48,shuffle=True) #This sets up a K-Fold cross-validation with 5 splits. The random_state parameter ensures reproducibility,

for trn_idx, test_idx in kf.split(trainX[columns],trainY): #This loop iterates over each split created by KFold
    X_tr,X_val=trainX[columns].iloc[trn_idx],trainX[columns].iloc[test_idx] #This splits trainX and trainY into training and validation sets based on the indices provided by KFold.
    y_tr,y_val=trainY.iloc[trn_idx],trainY.iloc[test_idx]
    model = RandomForestClassifier()
    model.fit(X_tr,y_tr)
    preds+=model.predict(testX[columns])/kf.n_splits

In [ ]:
preds2=[] #This initializes an empty list preds2 that will store the rounded predictions.
for p in preds: #This loops through each value p in the preds array.
    preds2+=[round(p,0)]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(testY, preds2, target_names=names, digits=4))

              precision    recall  f1-score   support

  Irrelevant     0.2829    0.1352    0.1830       318
    Negative     0.4422    0.4858    0.4630       599
     Neutral     0.2897    0.4723    0.3591       487
    Positive     0.5101    0.3389    0.4073       596

    accuracy                         0.3830      2000
   macro avg     0.3812    0.3581    0.3531      2000
weighted avg     0.4000    0.3830    0.3766      2000

